In [1]:
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
import numpy as np
import time

In [2]:
VOCAB_SIZE = 20000
BATCH_SIZE = 32
N_EPOCH = 2

In [3]:
def sparse_tfidf(X):
    t0 = time.time()
    count = np.zeros((len(X), VOCAB_SIZE))
    for i, indices in enumerate(X):
        for idx in indices:
            count[i, idx] += 1
    print("%.2f secs ==> Document-Term Matrix"%(time.time()-t0))

    t0 = time.time()
    X = TfidfTransformer().fit_transform(count)
    print("%.2f secs ==> TF-IDF transform\n"%(time.time()-t0))
    return X

In [4]:
def next_train_batch(X, y):
    for i in range(0, X.shape[0], BATCH_SIZE):
        yield X[i : i+BATCH_SIZE].toarray(), y[i: i+BATCH_SIZE]

def next_test_batch(X):
    for i in range(0, X.shape[0], BATCH_SIZE):
        yield X[i : i+BATCH_SIZE].toarray()

def train_input_fn(X_train, y_train):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(X_train, y_train), (tf.float32, tf.int64),
        (tf.TensorShape([None, VOCAB_SIZE]), tf.TensorShape([None])))
    iterator = dataset.make_one_shot_iterator()
    feats, labels = iterator.get_next()
    feats = {'x': feats}
    return feats, labels

def predict_input_fn(X_test):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_test_batch(X_test), tf.float32,
        tf.TensorShape([None, VOCAB_SIZE]))
    iterator = dataset.make_one_shot_iterator()
    feats = iterator.get_next()
    feats = {'x': feats}
    return feats

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=VOCAB_SIZE)
X_train = sparse_tfidf(X_train)
X_test = sparse_tfidf(X_test)

numeric_feat = tf.feature_column.numeric_column('x', shape=VOCAB_SIZE)

model = tf.estimator.DNNClassifier([50], [numeric_feat],
                                  dropout=0.5)

for _ in range(N_EPOCH):
    model.train(lambda: train_input_fn(X_train, y_train))
    y_pred = list(model.predict(lambda: predict_input_fn(X_test), predict_keys='class_ids'))
    y_pred = [kv['class_ids'][0] for kv in y_pred]
    print("\nValidation Accuracy: %.4f\n" % (y_pred==y_test).mean())

4.07 secs ==> Document-Term Matrix
9.86 secs ==> TF-IDF transform

3.39 secs ==> Document-Term Matrix
9.16 secs ==> TF-IDF transform

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp6mt7h0c3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x113ce39b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.